In [1]:
import os
import gc
import numpy as np
print(os.listdir('/cos_person/'))
import pandas as pd

['input', 'notebook', 'output']


In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
train = pd.read_csv('/cos_person/input/train_ver2.csv')
test = pd.read_csv('/cos_person/input/test_ver2.csv')
submission = pd.read_csv('/cos_person/input/sample_submission.csv')


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train=reduce_mem_usage(train)
test=reduce_mem_usage(test)
submission=reduce_mem_usage(submission)

Mem. usage decreased to 2342.72 Mb (53.1% reduction)
Mem. usage decreased to 122.34 Mb (28.1% reduction)
Mem. usage decreased to 10.64 Mb (25.0% reduction)


In [5]:
matrix=pd.concat([train, test], ignore_index=True, sort=False)
#填充缺失值，用什么填充还需待定
matrix.fillna(-999, inplace=True)
#drop掉没用的特征
matrix.drop(['fecha_alta','ult_fec_cli_1t','tipodom','cod_prov'],axis=1, inplace=True)

In [6]:
pd.set_option('display.max_columns', None)
matrix.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,0.0,6,1.0,1,A,S,N,-999,KHL,N,MALAGA,1.0,87218.1,02 - PARTICULARES,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-28,1050611,N,ES,V,23,0.0,35,1.0,1,I,S,S,-999,KHE,N,CIUDAD REAL,0.0,35548.7,03 - UNIVERSITARIO,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-28,1050612,N,ES,V,23,0.0,35,1.0,1,I,S,N,-999,KHE,N,CIUDAD REAL,0.0,122179,03 - UNIVERSITARIO,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-01-28,1050613,N,ES,H,22,0.0,35,1.0,1,I,S,N,-999,KHD,N,ZARAGOZA,0.0,119776,03 - UNIVERSITARIO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-28,1050614,N,ES,V,23,0.0,35,1.0,1,A,S,N,-999,KHE,N,ZARAGOZA,1.0,-999,03 - UNIVERSITARIO,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
matrix.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada',
       'indfall', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')

In [8]:
#将数据集按月份切割出来，d1512代表15年12月，d167代表16年7月，以此类推

matrix['fecha_dato']=pd.to_datetime(matrix['fecha_dato'],)
varia_list=locals()
for i in range(1,13):
       varia_list['d15'+str(i)]=matrix[(matrix.fecha_dato.dt.year==2015) & (matrix.fecha_dato.dt.month==i)]
for i in range(1,7):
       varia_list['d16'+str(i)]=matrix[(matrix.fecha_dato.dt.year==2016) & (matrix.fecha_dato.dt.month==i)]
del matrix
gc.collect()

15

In [9]:
#将上个月对应的值做为这个月的特征，对所有数据集进行处理
def get_last_month_feat(features=[]):
    if len(features)==0:
      return
#2015年
    for feat in features:
      for i in range(2,13):
         temp=varia_list['d15'+str(i-1)].loc[:,['ncodpers',feat]].rename(columns={feat:feat+'_last_month'}) 
         varia_list['d15'+str(i)] =  varia_list['d15'+str(i)].merge(temp, on='ncodpers', how='left')
#2016年
      for i in range(2,7):
         temp=varia_list['d16'+str(i-1)].loc[:,['ncodpers',feat]].rename(columns={feat:feat+'_last_month'}) 
         varia_list['d16'+str(i)] =  varia_list['d16'+str(i)].merge(temp, on='ncodpers', how='left')
#2015-2016年的转折
      temp=varia_list['d15'+str(12)].loc[:,['ncodpers',feat]].rename(columns={feat:feat+'_last_month'}) 
      varia_list['d16'+str(1)] =  varia_list['d16'+str(1)].merge(temp, on='ncodpers', how='left')

In [10]:
products=[ 'ind_cco_fin_ult1','ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

features=products+['ind_actividad_cliente','tiprel_1mes']


In [11]:
# 累计到上个月0-0，0-1，1-0，1-1模式转变的个数，对所有数据集进行处理，特别冗长，早知不把数据集的名字按年份分开命名
def count_index_change_pattern(features=[]):
   if len(features)==0:
     return
   for feat in features:
#2015年    
        for i in range(2,12):        
            varia_list['d15'+str(i)][feat+'_00_save']=np.zeros((varia_list['d15'+str(i)].shape[0],))
            varia_list['d15'+str(i)][feat+'_01_save']=np.zeros((varia_list['d15'+str(i)].shape[0],))
            varia_list['d15'+str(i)][feat+'_10_save']=np.zeros((varia_list['d15'+str(i)].shape[0],))
            varia_list['d15'+str(i)][feat+'_11_save']=np.zeros((varia_list['d15'+str(i)].shape[0],))
# COUNT 0 TO 0
            if feat+'_00' in varia_list['d15'+str(i)].columns:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==0),[feat+'_00_save']]= varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==0),][feat+'_00'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            else:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==0),[feat+'_00_save']]=1
            temp=varia_list['d15'+str(i)].loc[:,['ncodpers',feat+'_00_save']].rename(columns={feat+'_00_save':feat+'_00'}) 
            varia_list['d15'+str(i+1)] =  varia_list['d15'+str(i+1)].merge(temp, on='ncodpers', how='left')
# COUNT 0 TO 1
            if feat+'_01' in varia_list['d15'+str(i)].columns:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==1),[feat+'_01_save']]= varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==1),][feat+'_01'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            else:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==1),[feat+'_01_save']]=1
            temp=varia_list['d15'+str(i)].loc[:,['ncodpers',feat+'_01_save']].rename(columns={feat+'_01_save':feat+'_01'}) 
            varia_list['d15'+str(i+1)] =  varia_list['d15'+str(i+1)].merge(temp, on='ncodpers', how='left')       
# COUNT 1 TO 0
            if feat+'_10' in varia_list['d15'+str(i)].columns:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==1) & (varia_list['d15'+str(i)][feat]==0),[feat+'_10_save']]= varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==1) & (varia_list['d15'+str(i)][feat]==0),][feat+'_10'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            else:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==1) & (varia_list['d15'+str(i)][feat]==0),[feat+'_10_save']]=1
            temp=varia_list['d15'+str(i)].loc[:,['ncodpers',feat+'_10_save']].rename(columns={feat+'_10_save':feat+'_10'}) 
            varia_list['d15'+str(i+1)] =  varia_list['d15'+str(i+1)].merge(temp, on='ncodpers', how='left') 
# COUNT 1 TO 1
            if feat+'_11' in varia_list['d15'+str(i)].columns:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==1) & (varia_list['d15'+str(i)][feat]==1),[feat+'_11_save']]= varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==1) & (varia_list['d15'+str(i)][feat]==1),][feat+'_11'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            else:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==1) & (varia_list['d15'+str(i)][feat]==1),[feat+'_11_save']]=1
            temp=varia_list['d15'+str(i)].loc[:,['ncodpers',feat+'_11_save']].rename(columns={feat+'_11_save':feat+'_11'}) 
            varia_list['d15'+str(i+1)] =  varia_list['d15'+str(i+1)].merge(temp, on='ncodpers', how='left') 

#2015-2016年的转折            
        varia_list['d15'+str(12)][feat+'_00_save']=np.zeros((varia_list['d15'+str(12)].shape[0],))
        varia_list['d15'+str(12)][feat+'_01_save']=np.zeros((varia_list['d15'+str(12)].shape[0],))
        varia_list['d15'+str(12)][feat+'_10_save']=np.zeros((varia_list['d15'+str(12)].shape[0],))
        varia_list['d15'+str(12)][feat+'_11_save']=np.zeros((varia_list['d15'+str(12)].shape[0],))
# COUNT 0 TO 0
        if feat+'_00' in varia_list['d15'+str(12)].columns:
           varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==0) & (varia_list['d15'+str(12)][feat]==0),[feat+'_00_save']]= varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==0) & (varia_list['d15'+str(12)][feat]==0),][feat+'_00'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
        else:
           varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==0) & (varia_list['d15'+str(12)][feat]==0),[feat+'_00_save']]=1
        temp=varia_list['d15'+str(12)].loc[:,['ncodpers',feat+'_00_save']].rename(columns={feat+'_00_save':feat+'_00'}) 
        varia_list['d16'+str(1)] =  varia_list['d16'+str(1)].merge(temp, on='ncodpers', how='left')
# COUNT 0 TO 1
        if feat+'_01' in varia_list['d15'+str(12)].columns:
           varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==0) & (varia_list['d15'+str(12)][feat]==1),[feat+'_01_save']]= varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==1),][feat+'_01'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
        else:
           varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==0) & (varia_list['d15'+str(12)][feat]==1),[feat+'_01_save']]=1
        temp=varia_list['d15'+str(12)].loc[:,['ncodpers',feat+'_01_save']].rename(columns={feat+'_01_save':feat+'_01'}) 
        varia_list['d16'+str(1)] =  varia_list['d16'+str(1)].merge(temp, on='ncodpers', how='left')       
# COUNT 1 TO 0
        if feat+'_10' in varia_list['d15'+str(12)].columns:
           varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==1) & (varia_list['d15'+str(12)][feat]==0),[feat+'_10_save']]= varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==1) & (varia_list['d15'+str(i)][feat]==0),][feat+'_10'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
        else:
           varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==1) & (varia_list['d15'+str(12)][feat]==0),[feat+'_10_save']]=1
        temp=varia_list['d15'+str(12)].loc[:,['ncodpers',feat+'_10_save']].rename(columns={feat+'_10_save':feat+'_10'}) 
        varia_list['d16'+str(1)] =  varia_list['d16'+str(1)].merge(temp, on='ncodpers', how='left') 
# COUNT 1 TO 1
        if feat+'_11' in varia_list['d15'+str(12)].columns:
           varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==1) & (varia_list['d15'+str(12)][feat]==1),[feat+'_11_save']]= varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==1) & (varia_list['d15'+str(i)][feat]==1),][feat+'_11'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
        else:
           varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==1) & (varia_list['d15'+str(12)][feat]==1),[feat+'_11_save']]=1
        temp=varia_list['d15'+str(12)].loc[:,['ncodpers',feat+'_11_save']].rename(columns={feat+'_11_save':feat+'_11'}) 
        varia_list['d16'+str(1)] =  varia_list['d16'+str(1)].merge(temp, on='ncodpers', how='left') 

# 2016年
        for i in range(1,6):        
            varia_list['d16'+str(i)][feat+'_00_save']=np.zeros((varia_list['d16'+str(i)].shape[0],))
            varia_list['d16'+str(i)][feat+'_01_save']=np.zeros((varia_list['d16'+str(i)].shape[0],))
            varia_list['d16'+str(i)][feat+'_10_save']=np.zeros((varia_list['d16'+str(i)].shape[0],))
            varia_list['d16'+str(i)][feat+'_11_save']=np.zeros((varia_list['d16'+str(i)].shape[0],))
# COUNT 0 TO 0
            if feat+'_00' in varia_list['d16'+str(i)].columns:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==0) & (varia_list['d16'+str(i)][feat]==0),[feat+'_00_save']]= varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==0) & (varia_list['d16'+str(i)][feat]==0),][feat+'_00'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            else:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==0) & (varia_list['d16'+str(i)][feat]==0),[feat+'_00_save']]=1   
            temp=varia_list['d16'+str(i)].loc[:,['ncodpers',feat+'_00_save']].rename(columns={feat+'_00_save':feat+'_00'}) 
            varia_list['d16'+str(i+1)] =  varia_list['d16'+str(i+1)].merge(temp, on='ncodpers', how='left')

# COUNT 0 TO 1
            if feat+'_01' in varia_list['d16'+str(i)].columns:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==0) & (varia_list['d16'+str(i)][feat]==1),[feat+'_01_save']]= varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==0) & (varia_list['d16'+str(i)][feat]==1),][feat+'_01'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            else:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==0) & (varia_list['d16'+str(i)][feat]==1),[feat+'_01_save']]=1
            temp=varia_list['d16'+str(i)].loc[:,['ncodpers',feat+'_01_save']].rename(columns={feat+'_01_save':feat+'_01'}) 
            varia_list['d16'+str(i+1)] =  varia_list['d16'+str(i+1)].merge(temp, on='ncodpers', how='left')       
# COUNT 1 TO 0
            if feat+'_10' in varia_list['d16'+str(i)].columns:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==1) & (varia_list['d16'+str(i)][feat]==0),[feat+'_10_save']]= varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==1) & (varia_list['d16'+str(i)][feat]==0),][feat+'_10'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            else:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==1) & (varia_list['d16'+str(i)][feat]==0),[feat+'_10_save']]=1
            temp=varia_list['d16'+str(i)].loc[:,['ncodpers',feat+'_10_save']].rename(columns={feat+'_10_save':feat+'_10'}) 
            varia_list['d16'+str(i+1)] =  varia_list['d16'+str(i+1)].merge(temp, on='ncodpers', how='left') 
# COUNT 1 TO 1
            if feat+'_11' in varia_list['d16'+str(i)].columns:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==1) & (varia_list['d16'+str(i)][feat]==1),[feat+'_11_save']]= varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==1) & (varia_list['d16'+str(i)][feat]==1),][feat+'_11'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            else:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==1) & (varia_list['d16'+str(i)][feat]==1),[feat+'_11_save']]=1
            temp=varia_list['d16'+str(i)].loc[:,['ncodpers',feat+'_11_save']].rename(columns={feat+'_11_save':feat+'_11'}) 
            varia_list['d16'+str(i+1)] =  varia_list['d16'+str(i+1)].merge(temp, on='ncodpers', how='left') 
        for i in  range(2,13):
            varia_list['d15'+str(i)].drop([feat+'_00_save',feat+'_01_save',feat+'_10_save',feat+'_11_save'],axis=1, inplace=True)
        for i in range(1,6):
            varia_list['d16'+str(i)].drop([feat+'_00_save',feat+'_01_save',feat+'_10_save',feat+'_11_save'],axis=1, inplace=True)

          

In [12]:
#上个月买了几个产品,对所有数据集统一处理
products=[ 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1','ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

def get_n_products_last():
#2015年    
   for i in range(2,12): 
       varia_list['d15'+str(i)]['n_products_this']=np.zeros((varia_list['d15'+str(i)].shape[0],))
       for feat in products:
            varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==1),['n_products_this']]=varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat+'_last_month']==0) & (varia_list['d15'+str(i)][feat]==1),]['n_products_this'].apply(lambda x:int(x)+1) 
       temp=varia_list['d15'+str(i)].loc[:,['ncodpers','n_products_this']].rename(columns={'n_products_this':'n_products_last'}) 
       varia_list['d15'+str(i+1)] =  varia_list['d15'+str(i+1)].merge(temp, on='ncodpers', how='left') 
#2015-2016年的转折 
   varia_list['d15'+str(12)]['n_products_this']=np.zeros((varia_list['d15'+str(12)].shape[0],))
   for feat in products:
        varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==0) & (varia_list['d15'+str(12)][feat]==1),['n_products_this']]=varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat+'_last_month']==0) & (varia_list['d15'+str(12)][feat]==1),]['n_products_this'].apply(lambda x:int(x)+1) 
   temp=varia_list['d15'+str(12)].loc[:,['ncodpers','n_products_this']].rename(columns={'n_products_this':'n_products_last'}) 
   varia_list['d16'+str(1)] =  varia_list['d16'+str(1)].merge(temp, on='ncodpers', how='left') 
#2016年
   for i in range(1,6): 
       varia_list['d16'+str(i)]['n_products_this']=np.zeros((varia_list['d16'+str(i)].shape[0],))
       for feat in products:
            varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==0) & (varia_list['d16'+str(i)][feat]==1),['n_products_this']]=varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat+'_last_month']==0) & (varia_list['d16'+str(i)][feat]==1),]['n_products_this'].apply(lambda x:int(x)+1) 
       temp=varia_list['d16'+str(i)].loc[:,['ncodpers','n_products_this']].rename(columns={'n_products_this':'n_products_last'}) 
       varia_list['d16'+str(i+1)] =  varia_list['d16'+str(i+1)].merge(temp, on='ncodpers', how='left')
#drop掉那些只用来存储中间值的特征
   for i in range(2,13):
       varia_list['d15'+str(i)].drop(['n_products_this'],axis=1, inplace=True)
   for i in range(1,6):
       varia_list['d16'+str(i)].drop(['n_products_this'],axis=1, inplace=True)

In [13]:
#length of continuous 0 index until last month，截止到上个月为止共有几个0，对所有数据集统一处理
def get_length_conti0(features=[]):
    if len(features)==0:
      return
    for feat in features:
#2015年        
        for i in range(1,12): 
            varia_list['d15'+str(i)][feat+'_0len_save']=np.zeros((varia_list['d15'+str(i)].shape[0],))
            if feat+'_0len' in  varia_list['d15'+str(i)].columns:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat]==0),[feat+'_0len_save']]=varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat]==0),][feat+'_0len'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat]==1),[feat+'_0len_save']]=0
            else:
               varia_list['d15'+str(i)].loc[(varia_list['d15'+str(i)][feat]==0),[feat+'_0len_save']]=1
            temp=varia_list['d15'+str(i)].loc[:,['ncodpers',feat+'_0len_save']].rename(columns={feat+'_0len_save':feat+'_0len'}) 
            varia_list['d15'+str(i+1)] =  varia_list['d15'+str(i+1)].merge(temp, on='ncodpers', how='left')  
#2015-2016年的转折 
        varia_list['d15'+str(12)][feat+'_0len_save']=np.zeros((varia_list['d15'+str(12)].shape[0],))
        if feat+'_0len' in  varia_list['d15'+str(12)].columns:
            varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat]==0),[feat+'_0len_save']]=varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat]==0),][feat+'_0len'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
            varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat]==1),[feat+'_0len_save']]=0
        else:
            varia_list['d15'+str(12)].loc[(varia_list['d15'+str(12)][feat]==0),[feat+'_0len_save']]=1
        temp=varia_list['d15'+str(12)].loc[:,['ncodpers',feat+'_0len_save']].rename(columns={feat+'_0len_save':feat+'_0len'}) 
        varia_list['d16'+str(1)] =  varia_list['d16'+str(1)].merge(temp, on='ncodpers', how='left')  
#2016年
        for i in range(1,6): 
            varia_list['d16'+str(i)][feat+'_0len_save']=np.zeros((varia_list['d16'+str(i)].shape[0],))
            if feat+'_0len' in  varia_list['d16'+str(i)].columns:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat]==0),[feat+'_0len_save']]=varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat]==0),][feat+'_0len'].apply(lambda x:1 if pd.isnull(x) else int(x)+1)
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat]==1),[feat+'_0len_save']]=0
            else:
               varia_list['d16'+str(i)].loc[(varia_list['d16'+str(i)][feat]==0),[feat+'_0len_save']]=1
            temp=varia_list['d16'+str(i)].loc[:,['ncodpers',feat+'_0len_save']].rename(columns={feat+'_0len_save':feat+'_0len'}) 
            varia_list['d16'+str(i+1)] =  varia_list['d16'+str(i+1)].merge(temp, on='ncodpers', how='left')    
#drop掉那些只用来存储中间值的特征
        for i in range(1,13):
            varia_list['d15'+str(i)].drop([feat+'_0len_save'],axis=1, inplace=True)
        for i in range(1,6):
            varia_list['d16'+str(i)].drop([feat+'_0len_save'],axis=1, inplace=True)

In [14]:
#这个月是否有买了相应产品，将其作为一列，1为买了，0为没买,届时是作为训练用的标签，标签名为feat+'_new_purchase'
#df是以月份切割出的数据集
def pruchase_or_not(df,features=[]):
  for feat in features:  
   
       df[feat+'_new_purchase']=np.zeros((df.shape[0],))
       df.loc[(df[feat+'_last_month']==0) & (df[feat]==1),[feat+'_new_purchase']]=1


In [15]:
#target需要是字符串型,代表用哪个target来做mean value of the target variable
#df1是个用月份切割出来的数据集，用该数据集的mean value of the target variable作为
#df2数据集对应concaten_product特征的替代
def concaten_product_last_month(target,df1,df2):
    if  target+'_new_purchase' not in df1.columns:   
        pruchase_or_not(df1,features=[target]) 
        
    products_last_month=[ 
           'ind_cco_fin_ult1_last_month', 'ind_cder_fin_ult1_last_month',
           'ind_cno_fin_ult1_last_month', 'ind_ctju_fin_ult1_last_month',
           'ind_ctma_fin_ult1_last_month', 'ind_ctop_fin_ult1_last_month',
           'ind_ctpp_fin_ult1_last_month', 'ind_dela_fin_ult1_last_month',
           'ind_ecue_fin_ult1_last_month', 'ind_fond_fin_ult1_last_month',
           'ind_hip_fin_ult1_last_month', 'ind_plan_fin_ult1_last_month',
           'ind_pres_fin_ult1_last_month', 'ind_reca_fin_ult1_last_month',
           'ind_tjcr_fin_ult1_last_month', 'ind_valo_fin_ult1_last_month',
           'ind_viv_fin_ult1_last_month', 'ind_nomina_ult1_last_month',
           'ind_nom_pens_ult1_last_month', 'ind_recibo_ult1_last_month']
    

    temp=df1.groupby(products_last_month)[target+'_new_purchase'].value_counts().unstack().reset_index().rename(columns={0:'no', 1:'yes'})
    temp['yes']=temp['yes'].apply(lambda x:0 if pd.isnull(x) else int(x))
    temp['no']=temp['no'].apply(lambda x:0 if pd.isnull(x) else int(x))
    temp['concaten_product_target_mean']=temp['yes']/(temp['yes']+temp['no'])
    temp.drop(['yes','no'],axis=1, inplace=True)
   
 
    df1['concaten_product_target_mean']= df1.merge(temp, on=products_last_month, how='left')['concaten_product_target_mean']
    df2['concaten_product_target_mean']= df2.merge(temp, on=products_last_month, how='left')['concaten_product_target_mean']
     

In [16]:
#将这个月与上个月对应特征的特征值concaten后用相应的mean value of the target variable代替
#这里传入的feats按作者的想法是只有'indactividadcliente'和'tiprel_1mes'两种
#df1是个用月份切割出来的数据集，用该数据集的mean value of the target variable作为
#df2数据集concaten_feat特征的替代
def concaten_now_last_month(target,df1,df2,feats=[]):
  df1,df2
  if  target+'_new_purchase' not in df1.columns:   
     pruchase_or_not(df1,features=[target]) 
    
  for feat in feats:
     temp=df1.groupby([feat,feat+'_last_month'])[target+'_new_purchase'].value_counts().unstack().reset_index().rename(columns={0:'no', 1:'yes'})
     temp['yes']=temp['yes'].apply(lambda x:0 if pd.isnull(x) else int(x))
     temp['no']=temp['no'].apply(lambda x:0 if pd.isnull(x) else int(x))
     temp['concaten_'+feat+'_target_mean']=temp['yes']/(temp['yes']+temp['no'])
     temp.drop(['yes','no'],axis=1, inplace=True)

     df1['concaten_'+feat+'_target_mean']=df1.merge(temp, on=[feat,feat+'_last_month'], how='left')['concaten_'+feat+'_target_mean']
     df2['concaten_'+feat+'_target_mean']=df2.merge(temp, on=[feat,feat+'_last_month'], how='left')['concaten_'+feat+'_target_mean']              

In [17]:
products=[ 'ind_cco_fin_ult1',  'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
features=products+['ind_actividad_cliente','tiprel_1mes']
#构建数据集，
def build_dataset_for_all():
    get_last_month_feat(features)
    count_index_change_pattern(products)
    get_n_products_last()
    get_length_conti0(products)
    
#以上是对所有数据集统一处理的部分,可以先写到csv中
#可以拆开一条一条运行做测试    

    

In [18]:
 #这里仅用做测试
    get_last_month_feat(features)

In [19]:
#df1作为训练集，df2作为验证集or测试集，target是选择哪个product作为标签，作者的想法
#是训练一堆二分类器，最后把结果融合起来
#比如build_dataset_for_each('ind_valo_fin_ult1',d165,d166),即是以16年5月28日作为训练集，
#16年5月28日作为测试集构建数据集，'ind_valo_fin_ult1_new_purchase'作为标签，1代表买了，0代表没买

def build_dataset_for_each(target,df1,df2,):
   concaten_product_last_month(target,df1,df2,)
#feats按作者的想法是只有'ind_actividad_cliente'和'tiprel_1mes'两种
   concaten_now_last_month(target,df1,df2,feats=['ind_actividad_cliente','tiprel_1mes'])
#concaten完'ind_actividad_cliente_last_month','tiprel_1mes_last_month'就不需要了
   df1.drop(['ind_actividad_cliente_last_month','tiprel_1mes_last_month'],axis=1, inplace=True)
   df2.drop(['ind_actividad_cliente_last_month','tiprel_1mes_last_month'],axis=1, inplace=True)

In [20]:
#以下是用2015-12-28作为训练集，2016-06-28作为测试集,预测'ind_cco_fin_ult1'被购买的概率，但这只是生成数据集，后面还要结合lightgbm等进行训练
#build_dataset_for_all()
#以下用于测试
build_dataset_for_each('ind_cco_fin_ult1',d1512,d166)


#再剔除掉新用户(上个月没有记录)以及买过相应产品的用户（上个月已经买了这个产品）
d1512=d1512[(d1512['ind_cco_fin_ult1'+'_last_month']==0)]
d166=d166[(d166['ind_cco_fin_ult1'+'_last_month']==0)]
#这里还没封装好，仅先做测试

#用于Xgboost等工具直接训练的数据集还须再稍微处理下，还须特征的归特征，标签的归标签！！！


In [21]:
d1512.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada',
       'indfall', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1',
       'ind_cco_fin_ult1_last_month', 'ind_cder_fin_ult1_last_month',
       'ind_cno_fin_ult1_last_month', 'ind_ctju_fin_ult1_last_month',
       'ind_ctma_fi

In [24]:
d166.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada',
       'indfall', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1',
       'ind_cco_fin_ult1_last_month', 'ind_cder_fin_ult1_last_month',
       'ind_cno_fin_ult1_last_month', 'ind_ctju_fin_ult1_last_month',
       'ind_ctma_fi